In [68]:
import pandas as pd

In [69]:
test_df=pd.read_excel('data_set/Test_set.xlsx')

In [70]:
# test_df=test_df1.iloc[:2]

In [71]:
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [72]:
test_df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [73]:
test_df.shape

(2671, 10)

In [74]:
test_df.describe()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
count,2671,2671,2671,2671,2671,2671,2671,2671,2671,2671
unique,11,44,5,6,100,199,704,320,5,6
top,Jet Airways,9/05/2019,Delhi,Cochin,DEL → BOM → COK,10:00,19:00,2h 50m,1 stop,No info
freq,897,144,1145,1145,624,62,113,122,1431,2148


In [82]:
test_df['Airline'].unique()

array(['Jet Airways', 'IndiGo', 'Multiple carriers', 'Air Asia',
       'Air India', 'Vistara', 'SpiceJet', 'Vistara Premium economy',
       'GoAir', 'Multiple carriers Premium economy',
       'Jet Airways Business'], dtype=object)

In [89]:
a=['Jet Airways', 'IndiGo', 'Multiple carriers', 'Air Asia',
       'Air India', 'Vistara', 'SpiceJet', 'Vistara Premium economy',
       'GoAir', 'Multiple carriers Premium economy',
       'Jet Airways Business']

In [90]:
b=['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet']

In [92]:
for i in b:
    if i not in a:
        print(i)

Trujet


In [75]:
# All preprocessing steps done during training..

In [76]:
Airline_values=pd.get_dummies(test_df['Airline'], drop_first=True)
test_df['journey_day']=pd.to_datetime(test_df['Date_of_Journey'], format="%d/%m/%Y").dt.day
test_df['journey_month']=pd.to_datetime(test_df['Date_of_Journey'], format="%d/%m/%Y").dt.month
Source_value=pd.get_dummies(test_df['Source'], drop_first=True)
Destination_value=pd.get_dummies(test_df['Destination'], drop_first=True)
test_df['Dept_hr']=pd.to_datetime(test_df['Dep_Time']).dt.hour
test_df['Dept_min']=pd.to_datetime(test_df['Dep_Time']).dt.minute
test_df['Arrival_hr']=pd.to_datetime(test_df['Arrival_Time']).dt.hour
test_df['Arrival_min']=pd.to_datetime(test_df['Arrival_Time']).dt.minute

# Time taken by plane to reach destination is called Duration
# It is the differnce betwwen Departure Time and Arrival time


# Assigning and converting Duration column into list
duration = list(test_df["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only minutes from duration
    
# Adding duration_hours and duration_mins list to train_data dataframe

test_df["Duration_hours"] = duration_hours
test_df["Duration_mins"] = duration_mins

test_df.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)
# Concatinate Airline + Source + Destination
Data_test = pd.concat([test_df, Airline_values, Source_value, Destination_value], axis=1)
Data_test.drop(["Airline", "Date_of_Journey", "Source", "Destination", "Route", 
                "Dep_Time", "Arrival_Time", "Duration", "Additional_Info"], axis = 1, inplace = True)
# Data_train.head(2)






In [100]:
Data_test.head()

,Total_Stops,journey_day,journey_month,Dept_hr,Dept_min,Arrival_hr,Arrival_min,Duration_hours,Duration_mins,Air India,...,Chennai,Delhi,Kolkata,Mumbai,Cochin,Delhi,Hyderabad,Kolkata,New Delhi,Trujet
0,1,6,6,17,30,4,25,10,55,0,...,0,1,0,0,1,0,0,0,0,0
1,1,12,5,6,20,10,20,4,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,21,5,19,15,19,0,23,45,0,...,0,1,0,0,1,0,0,0,0,0
3,1,21,5,8,0,21,0,13,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,24,6,23,55,2,45,2,50,0,...,0,0,0,0,0,1,0,0,0,0


In [101]:
Data_test['Trujet']=0 # Beacuse one air line is missing in the input. So Adding to avoid error

# Loading the saved model

In [96]:
import pickle

model=pickle.load(open('Flight_RF_model.pkl', 'rb'))           

In [97]:
pred=model.predict(Data_test)

In [98]:
pred

array([13907.83028426,  4690.65279973, 15425.72246701, ...,
       16248.30996589, 12580.85028587,  7571.81255514])